Рекомендательные системы - это модели классификации, где целевая переменная - уровень интереса, а в качестве признаков используются данные по пользователю $x_{user}$ и товару $x_{item}$, а также (опционально) набор других признаков $x_{context}$.

Поэтому начнем обзор с моделей классификации.

# Модели

В наиболее общей формулировке модель машинного обучения выгллядит так:

$$y=f(x)$$
где y - целевая переменная, а x - вектор признаков.

Чаще всего ограничиваются классом линейных моделей:

$$y=w_0+w_1x_1+\cdots+w_nx_n$$ 

Далее для краткости я буду обозначать данную линейную комбинацию в виде скалярного произведения двух векторов $x$ и $w$ (свободный коэффициент $w_0$ также включаем в вектор w):

$$\hat{y}=\langle w,x \rangle$$

Для того чтобы модель лучше отслеживала нелинейные признаковые зависимости, можем в число признаков добавить попарные взаимодействия признаков (interactions), то есть слагаемые вида $x_i \cdot x_j$:

$$\hat{y}=\langle w,x\rangle + \sum_{i<j}w_{ij}x_ix_j$$

Помимо парных интеракций (2-order interactions), можно использовать и комбинации больших размерностей, однако на практике это редко используется.

-----

### Factorization Machines 

https://cseweb.ucsd.edu/classes/fa17/cse291-b/reading/Rendle2010FM.pdf

FM - метод, который слегка модифицирует линейную модель с интеракциями:

$$\hat{y}=\langle w,x\rangle +\sum_{i,j}\langle e_i,e_j\rangle x_ix_j$$

Видим, что вместо отдельного параметра $w_{ij}$ под каждую интеракцию мы используем его оценку, посчитанную как произведение латентных представлений каждого признака $\langle e_i,e_j\rangle$. В результате нам не нужно считать целиком матрицу $W$, а достаточно посчитать $n$ латентных представлений признаков.

Здесь и далее $e_i = e(x_i)$ - параметризованное латентное представление категориального признака $x_i$. В терминах нейронных сетей такое представление называется embedding и считается как $e(x_i)=x_iW_i$, где $W_i$ - некоторая матрица коэффициентов.

-----

### Matrix Factorization

FM не стоит путать с широко используемым в рекомендательных системах подходом MF (Matrix Factorization), в рамках которого целевая переменная $y=f(x_{user},x_{item})$ раскладывается в произведение латентных описаний двух переменных $x_{user}$ и $x_{item}$. По аналогии с используемой выше нотацией соотвествующую модель матричного разложения можно записать так:

$\hat{y} = \langle e_{user},e_{item}\rangle$

где $e_{user}=e(x_{user})$, $e_{item}=e(x_{item})$ - латентные сокращенные описания (они же эмбединги) признаков модели.

-----

### Wide-n-Deep (2016)

В 2016 году Google презентовали архитектуру сети, которую назвали Wide'n'Deep
$$\hat{y} = \sigma{(\alpha_{deep}f_{dnn}(e_1 \cdots e_n)) + \alpha_{wide}\sum_{i,j}w_{ij}x_ix_j})$$

Ключевая идея - зависимость целевой переменной от признаков $y=f(x)$ можно декомпозировать на
- "wide" часть, которая отвечает за запоминание 
- "deep" часть, которая отвечает за обобщение

Deep часть строится как обычная сеть (DNN, Dense Nueral Network), которой на вход подаются эмбединги признаков:
<IMG SRC="IMG/deep.png" width=250>


Wide часть - это просто набор интеракций признаков:
<img src="img/wide.png" width=250>


В модели Wide'n'Deep эти части объединяются логистической регрессией:

<img src="img/widendeep.png" width=550>

-----

### DeepFM (2017):

https://arxiv.org/abs/1703.04247

Отличие архитектуры от модели wide-and-deep в том, что вместо $w_{ij}$ используется слой факторизации:

$$\hat{y} = \sigma{(\alpha_{deep}f_{dnn}(e_1 \cdots e_n)) + \alpha_{wide}\sum_{i,j}{\langle e_i,e_j\rangle}x_ix_j})$$

<img src="img/deepfm.png" width=500>

-----

### xDeepFM (2018):

https://arxiv.org/abs/1803.05170

<img src="img/xdeepfm.png" width=500>

WIP

-----

### Neural Collaborative Filtering (NCF):

https://arxiv.org/pdf/1708.05031.pdf

Комбинация классического матричного разложения и глубокой модели на латентных описаниях.

<img src="img/ncf.png" width=500>

Здесь по $x_{user}$ и по $x_{item}$ генерируется по 2 эмбединга - один (MLP vector) для глубокой сети (MLP), другой (MF vector) для матрицчного разложения (GMF layer). Затем результаты конкатенирурются и подаются на вход логистической модели.

# Практика

### DeepFM

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ratings = pd.read_csv('./dataset/ml-1m/ratings.dat',sep='::', header=None, engine='python', names=['uid','mid','rating','timestamp'])
movies = pd.read_csv('./dataset/ml-1m/movies.dat',sep='::', header=None, engine='python', names=['mid','movie_name','movie_genre'])
users = pd.read_csv('./dataset/ml-1m/users.dat',sep='::', header=None, engine='python', names=['uid','user_fea1','user_fea2','user_fea3','user_fea4'])

tokenizer = Tokenizer(lower=True, split='|',filters='', num_words=15)
tokenizer.fit_on_texts(movies.movie_genre.values)
seq = tokenizer.texts_to_sequences(movies.movie_genre.values)
movies['movie_genre'] = pad_sequences(seq, maxlen=3,padding='post').tolist()
ratings = ratings.join(movies.set_index('mid'), on = 'mid', how = 'left')
ratings = ratings.join(users.set_index('uid'), on = 'uid', how = 'left')

# ----------------------

import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

def Tensor_Mean_Pooling(name = 'mean_pooling', keepdims = False):
    return Lambda(lambda x: K.mean(x, axis = 1, keepdims=keepdims), name = name)

def fm_1d(inputs, n_uid, n_mid, n_genre):
    
    fea3_input, uid_input, mid_input, genre_input = inputs
    
    # all tensors are reshape to (None, 1)
    num_dense_1d = [Dense(1, name = 'num_dense_1d_fea4')(fea3_input)]
    cat_sl_embed_1d = [Embedding(n_uid + 1, 1, name = 'cat_embed_1d_uid')(uid_input),
                        Embedding(n_mid + 1, 1, name = 'cat_embed_1d_mid')(mid_input)]
    cat_ml_embed_1d = [Embedding(n_genre + 1, 1, mask_zero=True, name = 'cat_embed_1d_genre')(genre_input)]

    cat_sl_embed_1d = [Reshape((1,))(i) for i in cat_sl_embed_1d]
    cat_ml_embed_1d = [Tensor_Mean_Pooling(name = 'embed_1d_mean')(i) for i in cat_ml_embed_1d]
    
    # add all tensors
    y_fm_1d = Add(name = 'fm_1d_output')(num_dense_1d + cat_sl_embed_1d + cat_ml_embed_1d)
    
    return y_fm_1d

def fm_2d(inputs, n_uid, n_mid, n_genre, k):
    
    fea3_input, uid_input, mid_input, genre_input = inputs
    
    num_dense_2d = [Dense(k, name = 'num_dense_2d_fea3')(fea3_input)] # shape (None, k)
    num_dense_2d = [Reshape((1,k))(i) for i in num_dense_2d] # shape (None, 1, k)

    cat_sl_embed_2d = [Embedding(n_uid + 1, k, name = 'cat_embed_2d_uid')(uid_input), 
                       Embedding(n_mid + 1, k, name = 'cat_embed_2d_mid')(mid_input)] # shape (None, 1, k)
    
    cat_ml_embed_2d = [Embedding(n_genre + 1, k, name = 'cat_embed_2d_genre')(genre_input)] # shape (None, 3, k)
    cat_ml_embed_2d = [Tensor_Mean_Pooling(name = 'cat_embed_2d_genure_mean', keepdims=True)(i) for i in cat_ml_embed_2d] # shape (None, 1, k)

    # concatenate all 2d embed layers => (None, ?, k)
    embed_2d = Concatenate(axis=1, name = 'concat_embed_2d')(num_dense_2d + cat_sl_embed_2d + cat_ml_embed_2d)

    # calcuate the interactions by simplication
    # sum of (x1*x2) = sum of (0.5*[(xi)^2 - (xi^2)])
    tensor_sum = Lambda(lambda x: K.sum(x, axis = 1), name = 'sum_of_tensors')
    tensor_square = Lambda(lambda x: K.square(x), name = 'square_of_tensors')

    sum_of_embed = tensor_sum(embed_2d)
    square_of_embed = tensor_square(embed_2d)

    square_of_sum = Multiply()([sum_of_embed, sum_of_embed])
    sum_of_square = tensor_sum(square_of_embed)

    sub = Subtract()([square_of_sum, sum_of_square])
    sub = Lambda(lambda x: x*0.5)(sub)
    y_fm_2d = Reshape((1,), name = 'fm_2d_output')(tensor_sum(sub))
    
    return y_fm_2d, embed_2d

def deep_part(embed_2d, dnn_dim, dnn_dr):
    
    # flat embed layers from 3D to 2D tensors
    y_dnn = Flatten(name = 'flat_embed_2d')(embed_2d)
    for h in dnn_dim:
        y_dnn = Dropout(dnn_dr)(y_dnn)
        y_dnn = Dense(h, activation='relu')(y_dnn)
    y_dnn = Dense(1, activation='relu', name = 'deep_output')(y_dnn)
    
    return y_dnn


# Model Parameters
n_uid = ratings.uid.max()
n_mid= ratings.mid.max()
n_genre=14
k=20
dnn_dim=[64,64]
dnn_dr=0.5
    
# numerica features
fea3_input = Input((1,), name = 'input_fea3')
num_inputs = [fea3_input]
# single level categorical features
uid_input = Input((1,), name = 'input_uid')
mid_input = Input((1,), name= 'input_mid')
cat_sl_inputs = [uid_input, mid_input]

# multi level categorical features (with 3 genres at most)
genre_input = Input((3,), name = 'input_genre')
cat_ml_inputs = [genre_input]

inputs = num_inputs + cat_sl_inputs + cat_ml_inputs
    
# Define subnets
y_fm_1d = fm_1d(inputs, n_uid, n_mid, n_genre)
y_fm_2d, embed_2d = fm_2d(inputs, n_uid, n_mid, n_genre, k)
y_dnn = deep_part(embed_2d, dnn_dim, dnn_dr)
    
# combinded deep and fm parts
y = Concatenate()([y_fm_1d, y_fm_2d, y_dnn])
y = Dense(1, name = 'deepfm_output')(y)
    
fm_model_1d = Model(inputs, y_fm_1d)
fm_model_2d = Model(inputs, y_fm_2d)
deep_model = Model(inputs, y_dnn)
deep_fm_model = Model(inputs, y)

# -------------------------------------------------------


# Format Dataset
def df2xy(ratings):
    x = [ratings.user_fea3.values, 
         ratings.uid.values, 
         ratings.mid.values, 
         np.concatenate(ratings.movie_genre.values).reshape(-1,3)]
    y = ratings.rating.values
    return x,y

in_train_flag = np.random.random(len(ratings)) <= 0.9
train_data = ratings.loc[in_train_flag,]
valid_data = ratings.loc[~in_train_flag,]
train_x, train_y = df2xy(train_data)
valid_x, valid_y = df2xy(valid_data)

# train  model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
deep_fm_model.compile(loss = 'MSE', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model_ckp = ModelCheckpoint(filepath='./model/deepfm_weights.h5', 
                            monitor='val_loss',
                            save_weights_only=True, 
                            save_best_only=True)
callbacks = [model_ckp,early_stop]
train_history = deep_fm_model.fit(train_x, train_y, 
                                  epochs=30, batch_size=2048, 
                                  validation_split=0.1, 
                                  callbacks = callbacks)



### Neural Collaborative Filtering

Попробуем релаизовать подход NCF с помощью библиотеки Keras. В качестве обучающей выборки используем датасет MovieLens.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

dataset = pd.read_csv("/Users/nipun/Downloads/ml-100k/u.data",sep='\t',names="user_id,item_id,rating,timestamp".split(","))

dataset.user_id = dataset.user_id.astype('category').cat.codes.values
dataset.item_id = dataset.item_id.astype('category').cat.codes.values

# Train/Test
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2)

y_true = test.rating



import keras

# Model Parameters
n_latent_factors_user = 8
n_latent_factors_movie = 10
n_latent_factors_mf = 3
n_users, n_movies = len(dataset.user_id.unique()), len(dataset.item_id.unique())

movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding_mlp = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding-MLP')(movie_input)
movie_vec_mlp = keras.layers.Flatten(name='FlattenMovies-MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(n_movies + 1, n_latent_factors_mf, name='Movie-Embedding-MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name='FlattenMovies-MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.2)(movie_vec_mf)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec_mlp = keras.layers.Flatten(name='FlattenUsers-MLP')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding-MLP')(user_input))
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_vec_mf = keras.layers.Flatten(name='FlattenUsers-MF')(keras.layers.Embedding(n_users + 1, n_latent_factors_mf,name='User-Embedding-MF')(user_input))
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)


concat = keras.layers.merge([movie_vec_mlp, user_vec_mlp], mode='concat',name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = keras.layers.BatchNormalization(name='Batch')(dense)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name='Batch-2')(dense_2)


dropout_2 = keras.layers.Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = keras.layers.merge([movie_vec_mf, user_vec_mf], mode='dot',name='Dot')


pred_mlp = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)

combine_mlp_mf = keras.layers.merge([pred_mf, pred_mlp], mode='concat',name='Concat-MF-MLP')
result_combine = keras.layers.Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = keras.layers.Dense(100,name='FullyConnected-4')(result_combine)


result = keras.layers.Dense(1,name='Prediction')(deep_combine)


model = keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr =0.01)
model.compile(optimizer='adam',loss= 'mean_absolute_error')

Обучим модель и посмотрим, какое качество она показывает

In [ ]:
history = model.fit([train.user_id, train.item_id], train.rating, epochs=25, verbose=0, validation_split=0.1)

from sklearn.metrics import mean_absolute_error
y_hat_2 = np.round(model.predict([test.user_id, test.item_id]),0)
print(mean_absolute_error(y_true, y_hat_2))

print(mean_absolute_error(y_true, model.predict([test.user_id, test.item_id])))